## LAB 2 - COMMON CRAWL DATA COLLECTION
Code Reference:- https://www.bellingcat.com/resources/2015/08/13/using-python-to-mine-common-crawl/

#### Submitted by -
    Pradeep Kumar Joshi
    Abhishek Goswami

In [0]:
import requests
import argparse
import time
import json
from io import BytesIO
import gzip
import sys
import csv
import codecs
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [0]:
## Specifying the Common Crawl Index and Domain to fetch data
cc_index = ["2019-04","2019-09","2019-13"]
cc_domain = "americanimmigrationcouncil.org"

In [0]:
def compute(cc_domain):

    data_collect = []
    for index in cc_index:
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-"+index+"-index?" 
        cc_url += "url="+cc_domain+"&matchType=domain&output=json" 
        
        ack = requests.get(cc_url)
        if ack.status_code == 200:
            val_list = ack.content.splitlines()
            for val in val_list:
                data_collect.append(json.loads(val))
    return data_collect        

In [0]:
def scrappedData(info):

    offset, length = int(info['offset']), int(info['length'])
    offset_end = offset + length - 1

    url_append = 'https://commoncrawl.s3.amazonaws.com/'
    acknldge = requests.get(url_append + info['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

    info_collect = BytesIO(acknldge.content)
    info_unzipped = gzip.GzipFile(fileobj=info_collect)

    unzipped_info = info_unzipped.read()
    return unzipped_info

In [0]:
def fetchURL(hyper_text,url_fetched):

    parser = BeautifulSoup(hyper_text)
    loops = parser.find_all("a")
    if loops:
        for loop in loops:
            text_refer = loop.attrs.get("href")
            if text_refer is not None:
                if cc_domain not in text_refer:
                    if text_refer not in url_fetched and text_refer.startswith("http"):
                        url_fetched.append(text_refer)
    return url_fetched

In [0]:
### URL COLLECTION
def collectarticles_url(data_collect):
    url_fetch = []
    
    for i in data_collect['ack']['docs']:
        urls = {}  
        urls['url'] = i['web_url']
        url_fetched = urls['url']
        url_fetch.append(url_fetched)
        
    return(url_fetch)

In [88]:
data_collect = compute(cc_domain)
url_fetched   = []

for doc in data_collect:
    hyper_text = scrappedData(doc)
    url_fetched = fetchURL(hyper_text,url_fetched)
    
print("URLs fetched for Common Crawl: ", + len(url_fetched)) 

URLs fetched for Common Crawl:  1393


In [0]:
## SCRAPING THE DATA FROM URLs
cc_finalList = []
for i in url_fetched:
    try:
        req = urllib.request.urlopen(i).read()
    except urllib.error.HTTPError as e:
        continue
        
    soup = BeautifulSoup(req)
    loop_text = soup.findAll('p')
    
    data_text = ""
    
    for val in loop_text:
        data_text = data_text + val.getText()
    cc_finalList.append(data_text)

In [0]:
## Generating Common Crawl Text File
cc_immigDF = pd.DataFrame(cc_finalList)
cc_immigDF.to_csv('common_crawl.txt', index=False)